# **ArcPy Functions**

These are functions that I have used in previous scripts with ArcPy. Most function serve a very specific purpose, but some are generally pretty handy such as ```GetParameterFromName```.

## ```GetParameterFromName``` and ```GetParameterFromNameAsText```
Allows parameters to be set by the name designated within the script tool parameters pane, rather than by their assigned index number. This allows the UI and parameter order of toolbox scripts to be altered without needing to update indexes used throughout the script.

| Parameter | Explanation | Data Type |
| --- | --- | --- |
|param_name | The name of the parameter. | String |

### Set Parameter Index Number to Variable

In [ ]:

def GetParameterFromName(param_name):
    #Set IndexNum as a variable to be used elsewhere
    global IndexNum
    #Clear values from IndexNum (for subsequent uses of function)
    IndexNum = None
    #Get parameter info and assign index number
    params = arcpy.GetParameterInfo()
    for i, param in enumerate(params):
        if param.name == param_name:
            IndexNum = i

#Usage Example:
##First Example
GetParameterFromName("Parameter_Name")
GetParameter(IndexNum)

##Second Example
GetParameterFromName("Parameter_Name")
GetParameterAsText(IndexNum)

### Replace ```GetParameter```

In [ ]:
def GetParamByName(param_name):
    #Clear values from i (for subsequent uses of function)
    i = None
    #Get parameter info and assign index number
    params = arcpy.GetParameterInfo()
    for i, param in enumerate(params):
        if param.name == param_name:
            arcpy.GetParameter(i)

#Usage Example:
##First Example
x = GetParamByName("Parameter_Name1")
y = GetParamByName("Parameter_Name2")

##Second Example
arcpy.function(GetParamByName("Parameter_Name"),"Text",x,y)

### Replace ```GetParameterAsText```

In [ ]:
def GetParamByNameAsText(param_name):
    #Clear values from i (for subsequent uses of function)
    i = None
    #Get parameter info and assign index number
    params = arcpy.GetParameterInfo()
    for i, param in enumerate(params):
        if param.name == param_name:
            arcpy.GetParameterAsText(i)

#Usage Example:
##First Example
x = GetParamByNameAsText("Parameter_Name1")
y = GetParamByNameAsText("Parameter_Name2")

##Second Example
arcpy.function(GetParamByNameAsText("Parameter_Name"),"Text",x,y)

## ```GenerateUniqueUUID```
Generates a unique UUID value for all blank cells in a given field. Utilizes uuid module.

| Parameter | Explanation | Data Type | 
| --- | --- | --- |
| in_FC | The input feature class. The field here is where values are added. | Feature Class |
| field_name | The field where UUID values will be added. | String |

In [ ]:
def GenerateUniqueUUID(in_FC, field_name):
        try:
            #Create an update cursor to update the field
            with arcpy.da.UpdateCursor(in_FC, [field_name]) as cursor:
                for row in cursor:
                    #Check if the cell is empty
                    if row[0] is None or row[0] is "<Null>" or row[0]== "":
                        # Update the cell with a unique UUID
                        row[0] = str(uuid.uuid4()) #Formatting to UUID string can be added here
                        cursor.updateRow(row)
        except Exception as e:
            print("Error occurred:", e)

#Usage Example:
import uuid

FeatureClass = GetParameter(1)
GenerateUniqueUUID(FeatureClass,"Field1")

## ```AssignBlankCells```
Assigns a specified value to all blank cells in a given field.
| Parameter | Explanation | Data Type |
| --- | --- | --- |
| in_FC | The input feature class. The field here is where values are added. | Feature Class |
| field_name | The field where values will be added. | String |
| value_to_add | Value that will be added to any blank cells. | String |

In [ ]:
def AssignBlankCells(in_FC, field_name, value_to_add):
    try:
        #Create an update cursor to update the field
        with arcpy.da.UpdateCursor(input_feature_class, [field_name]) as cursor:
            for row in cursor:
                #Check if the cell is empty
                if row[0] is None or row[0] is "<Null>" or row[0]== '':
                    #Update the cell with the specified value
                    row[0] = value_to_add
                    cursor.updateRow(row)
    except Exception as e:
        print("Error occurred:", e)

#Usage Example:
FeatureClass = GetParameter(1)
AssignBlankCells(FeatureClass,"Field1","Value")


## ```FieldCheckCreate``` ************************************
Creates a requested field if it does not exist.

| Parameter | Explanation | Data Type |
| --- | --- | --- |
| in_FC | The input feature class. Where the check and creation of a field occurs. | Feature Class |
| field_name | The field that will be checked and/or created | String |
| field_length | The length of the field if it is created | String |
| field_alias | The alias of the field if it is created | String |

In [ ]:
def FieldCheckCreate(in_FC, field_name, field_type, field_length, field_alias):
    try:
        arcpy.management.AddField(in_FC, field_name, field_type, "", "", field_length, field_alias)
    except arcpy.ExecuteError:
        #Field already exists, so skip creating it
        pass

#Usage Example:
FeatureClass = GetParameter(1)
FieldCheckCreate(FeatureClass,"Field1","TEXT","255","First Field")

## ```CRASCodedDomain```
Creates and assigns a coded domain to a given field. Utilizes NumPy module.

| Parameter | Explanation | Data Type |
| --- | --- | --- |
| workspace | The workspace that the domain will be applied to | Workspace |
| in_FC | The input feature class. Contains the field that the domain will be assigned to. | Feature Class |
| field_name | The field that the domain will be applied to | String |
| DOM_name | The name of the domain that will be created. | String |
| DOM_values | The values for domain codes and descriptions. | List |
| dtype1 | The NumPy data type that is used for domain code values. | String |
| dtype2 | The NumPy data type that is used for value description values. | String |

In [ ]:

def CRASCodedDomain(workspace,in_FC,field_name,DOM_name,DOM_desc,DOM_values,dtype1,dtype2):
    #Creaty an array and empty GDB table
    array = numpy.array(DOM_values, dtype=[('code', dtype1), ('desc', dtype2)])
    table = 'in_memory/table'
    #Convert array to GDB table
    arcpy.da.NumPyArrayToTable(array, table)
    #Convert GDB table to domain
    arcpy.management.TableToDomain(table, 'code', 'desc', workspace, DOM_name,DOM_desc)
    #Assign created domain to a field
    arcpy.management.AssignDomainToField(in_FC,field_name,DOM_name) #UNTESTED

#Usage Example:
import numpy

WS = GetParameter(1)
DomainValues = [(Code1, Desc1),(Code2,Desc2),(Code3,Desc3)]
CRASCodedDomain(WS,"Field1","Domain1","Description",DomainValues,"S5","S10")

https://gis.stackexchange.com/questions/228925/creating-domains-and-coded-values-using-arcpy